In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
#  Add the clean movie function that takes in the argument, "movie".
#
# This function is part of the execute step of the three-step process of data cleaning:
# .inspect / .plan / .execute
# The .inspect and .plan steps having been finished throughout Module 8, this notebook consists of
# primarily summation .execute steps; a descriptive walk-through of the full process is found in
# the Jupyter notebook file "ETL_StepThree.ipynb," in the repository, or of course in the online
# material for Module 8. Running this function on the (full) set of movies in the current Wikipedia
# file dataset pares what were 193 initial columns to 39 (which have been filtered and merged --
# as printed in commented-out line 39 of the `extract_transform_load()` function below in this
# notebook -- that line is as follows: `print(len(wiki_movies_df.columns))`)
# ... and the list of the 39 columns is:
# ['url', 'year', 'imdb_link', 'title', 'Based on', 'Starring', 'Narrated by', 'Cinematography',
# 'Release date', 'Running time', 'Country', 'Language', 'Budget', 'Box office', 'Director',
# 'Distributor', 'Editor(s)', 'Composer(s)', 'Producer(s)', 'Production company(s)', 'Writer(s)',
# 'Genre', 'Original language(s)', 'Original network', 'Executive producer(s)',
# 'Production location(s)', 'Picture format', 'Audio format', 'Voices of', 'Followed by',
# 'Created by', 'Preceded by', 'Suggested by', 'alt_titles', 'Recorded', 'Venue', 'Label',
# 'Color process', 'Animator(s)'].
#
# Below is a list of the initial 193 columns (found from D1 `ETL_function_test.ipynb` Step 4
# DataFrame, or commented-out line 34 [which is: `wiki_movies_df = pd.DataFrame(wiki_movies_raw)`]
# of the `extract_transform_load()` function below in *this* notebook) ...
# ['url', 'year', 'imdb_link', 'title', 'Directed by', 'Produced by', 'Screenplay by',
# 'Story by', 'Based on', 'Starring', 'Narrated by', 'Music by', 'Cinematography', 'Edited by',
# 'Productioncompany ', 'Distributed by', 'Release date', 'Running time', 'Country', 'Language',
# 'Budget', 'Box office', 'Written by', 'Genre', 'Theme music composer', 'Country of origin',
# 'Original language(s)', 'Producer(s)', 'Editor(s)', 'Production company(s)', 'Original network',
# 'Original release', 'Productioncompanies ', 'Executive producer(s)', 'Production location(s)',
# 'Distributor', 'Picture format', 'Audio format', 'Voices of', 'Followed by', 'Composer(s)',
# 'Created by', 'Also known as', 'Opening theme', 'No. of episodes', 'Preceded by', 'Author',
# 'Publisher', 'Publication date', 'Media type', 'Pages', 'ISBN', 'OCLC', 'LC Class',
# 'Cover artist', 'Series', 'Set in', 'Adaptation by', 'Suggested by', 'Biographical data',
# 'Born', 'Died', 'Resting place', 'Occupation', 'Years active', 'Spouse(s)', 'Children',
# 'Parent(s)', 'Genres', 'Instruments', 'Labels', 'Website', 'Traditional', 'Mandarin', 'Type',
# 'Industry', 'Fate', 'Founded', 'Founder', 'Headquarters', 'Parent', 'Released', 'Recorded',
# 'Venue', 'Length', 'Label', 'Director', 'Producer', 'Area', 'Coordinates', 'Status',
# 'Opening date', 'Closing date', 'Replaced', 'Replaced by', 'Name', 'Attraction type', 'Music',
# 'Duration', 'Simplified Chinese', 'Traditional Chinese', 'Hanyu Pinyin', 'Literal meaning',
# 'Transcriptions', 'Bopomofo', 'Gwoyeu Romatzyh', 'Wade–Giles', 'IPA', 'Yale Romanization',
# 'Jyutping', 'Hokkien POJ', 'Animation by', 'Color process', 'Engine(s)', 'Genre(s)',
# 'Actor control', 'Production company', 'Release(s)', 'Format(s)', 'Simplified', 'Characters',
# 'Date premiered', 'Place premiered', 'Setting', 'Original language', 'Subject', 'Published',
# 'Dewey Decimal', 'Text', 'Illustrator', 'Original title', 'Published in English', 'French',
# 'Developed by', 'Ending theme', 'No. of seasons', 'Nationality', 'Portrayed by', 'Alias',
# 'Species', 'Gender', 'Family', 'Alma mater', 'Camera setup', 'Novel(s)', 'Comics', 'Film(s)',
# 'Screen story by', 'Hangul', 'Revised Romanization', 'McCune–Reischauer', 'Developer(s)',
# 'Publisher(s)', 'Designer(s)', 'Programmer(s)', 'Artist(s)', 'Writer(s)', 'Engine',
# 'Platform(s)', 'Release', 'Mode(s)', 'Original work', 'Television series', 'Japanese',
# 'Hepburn', 'Literally', 'Cantonese', 'Full name', 'Height', 'Seasons', 'Chinese',
# 'Other names', 'Relatives', 'Yiddish', 'Formerly', 'Key people', 'Total assets', 'Owner', 
# 'Number of employees', 'Divisions', 'Subsidiaries', 'Arabic', 'Romanized', 'Predecessor',
# 'Founders', 'Area served', 'Products', 'Services', 'Russian', 'Hebrew', 'Revenue',
# 'Operating income', 'Polish']

def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    # make an empty dict to hold all of the alternative titles...
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as',
                'Arabic',
                'Cantonese',
                'Chinese',
                'French',
                'Hangul',
                'Hebrew',
                'Hepburn',
                'Japanese',
                'Literally',
                'Mandarin',
                'McCune–Reischauer',
                'Original title',
                'Polish',
                'Revised Romanization',
                'Romanized',
                'Russian',
                'Simplified',
                'Traditional',
                'Yiddish'\
                #'title'
               ]:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
    
    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)

    # following two columns of the 193 could be joined perhaps into a column called 'Narrator(s)'
    # but leaving two separate columns for now ...
    #  'Narrated by',
    #  'Voices of',
    #
    # Determine new names
    # two comment lines below concern column-new-name tally totals
    #  19(9)[5]{2} : to be changed(no change needed)[already new/keep as is]{to remove/combine later?}
    #   : red text(denoted as "already default")[all lowercase name]{left #comment, no notation}
    change_column_name( 'Adaptation by', 'Writer(s)' )
    change_column_name( 'Animation by', 'Animator(s)' )
#    change_column_name( 'Composer(s)', '?name')              #already default
#    change_column_name( 'Country', '?name')                  #already default
    change_column_name( 'Country of origin', 'Country' )
    change_column_name( 'Directed by', 'Director' )
#    change_column_name( 'Director', '?name')                 #already default

    change_column_name( 'Distributed by', 'Distributor' )
#     change_column_name( 'Distributed by', 'Distributor(s)' )  # use default in line above instead
#     change_column_name( 'Distributor', 'Distributor(s)' )  # make default instead of changing

    change_column_name( 'Edited by', 'Editor(s)' )
#    change_column_name( 'Editor(s)', '?name')                #already default
    change_column_name( 'Length', 'Running time' )
    change_column_name( 'Music by', 'Composer(s)' )
#    change_column_name( 'Narrated by', 'Narrator(s)' )
    change_column_name( 'Original release', 'Release date' )
    change_column_name( 'Produced by', 'Producer(s)' )
    change_column_name( 'Producer', 'Producer(s)' )
#    change_column_name( 'Producer(s)', '?name')              #already default
#    change_column_name( 'Production company(s)', '?name')    #already default
    change_column_name( 'Productioncompanies ', 'Production company(s)' )
    change_column_name( 'Productioncompany ', 'Production company(s)' )
#    change_column_name( 'Release date', '?name')             #already default
    change_column_name( 'Released', 'Release date' )
#    change_column_name( 'Running time', '?name')             #already default
    change_column_name( 'Screen story by', 'Writer(s)' )
    change_column_name( 'Screenplay by', 'Writer(s)' )
    change_column_name( 'Story by', 'Writer(s)' )
    change_column_name( 'Theme music composer', 'Composer(s)' )
#    change_column_name( 'Voices of', 'Narrator(s)' )
    change_column_name( 'Written by', 'Writer(s)' )

    return movie

In [3]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():  # function_name():
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
    ratings = pd.read_csv(ratings_file)

    # Open and read the Wikipedia data JSON file.
    with open(wiki_file, mode='r') as file:
        wiki_movies_raw = json.load(file)
        
    # D2-3. Write a list comprehension to filter out TV shows.
    #   # wiki_movies = [ movie for movie in wiki_movies_raw
    #   #                 if 'No. of episodes' not in movie ]

    #   # wiki_movies = [ movie for movie in wiki_movies_raw
    #   #                 if 'No. of episodes' not in movie
    #   #                 and ('Director' in movie or 'Directed by' in movie)
    #   #                 and 'imdb_link' in movie
    #   #               ]
    wiki_movies = [ movie for movie in wiki_movies_raw
                    if
                    ('Director' in movie or 'Directed by' in movie)
                    and 'imdb_link' in movie \
                    and 'No. of episodes' not in movie
                  ]

    # D2-4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_movies]

    # D2-5. Read in the cleaned movies list from D2-4 as a DataFrame.
    # wiki_movies_df = pd.DataFrame(wiki_movies_raw)
    wiki_movies_df = pd.DataFrame(clean_movies)

    #   print(f"len (raw):{len(wiki_movies_raw)}, len (pared):{len(clean_movies)};\nDataFrame (clean):\
    #       \n    len ... {len(wiki_movies_df)}\n    columns ... {len(wiki_movies_df.columns)}")
    #   # print(len(wiki_movies_df.columns))
    #   print(wiki_movies_df.columns.to_list())
    
    # D2-6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        # wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(
        #                     r'(tt\d{7})')

        # wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(
        #         r'(tt\d{7})')[0].drop_duplicates(subset=['imdb_id'], keep=False)

        # imdb_id = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')

        # wiki_movies_df['imdb_id'] = imdb_id[0].drop_duplicates(keep=False)

        # wiki_movies_df['imdb_id'] = (wiki_movies_df['imdb_link'].str.extract(
        #     r'(tt\d{7})')).drop_duplicates(keep=False)       

        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')       
        wiki_movies_df.drop_duplicates(subset=['imdb_id'], inplace=True)
        # wiki_movies_df.drop_duplicates(subset=['imdb_id'], keep=False, inplace=True)
                
        #code using lambda function modified from Shift-Tab info. for .loc property
        #has_duplicate = len(
        #    wiki_movies_df.loc[lambda df: df['imdb_id'] == string_check]) > 1
        
        #if has_duplicate:
        #    print(f"already existing 'imdb_id' = '{string_check}', ... skipping.")
        #else:
        #    wiki_movies_df['imdb_id'] = string_check       
    except AttributeError:
        print(f"An error (AttributeError) occurred.")
    except KeyError: 
        print(f"An error (KeyError) occurred.")
    except SyntaxError:
        print("An error (SyntaxError) occurred.")
    except TypeError:
        print("An error (TypeError) occurred.")
    # except ValueError:
    #     print("A ValueError occurred.")
    
    #  D2-7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    # wiki_columns_to_keep = [column for column in wiki_movies_df.columns\
    #       if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]

    # wiki_columns_to_keep =\
    #     [column for column in wiki_movies_df.columns\
    #         if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df)]

    # keep columns that are 10% or more non-null values (less than 90% is null)
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns\
          if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]

    # print(f'{wiki_movies_df[wiki_columns_to_keep].isnull().sum()}')
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
    
    # D2-8. Create a variable that will hold the non-null values from the “Box office” column.
    #has_box_office = (wiki_movies_df['Box office'].isnull() == False)
    box_office = wiki_movies_df['Box office'].dropna()

    # D2-9. Convert the box office data created in D2-8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # D2-10. Write a regular expression to match the six elements of "form_one" of the box office data.
    #   # "Create the First Form ['$123.4 million' (or billion) -- Module 8.3.10]
    #   # For the first form (f1 comments below), our pattern match string will include six elements
    #   # in the following order:
    #   #
    #f1 # f1_1. A dollar sign
    #f1 # f1_2. An arbitrary (but non-zero) number of digits
    #f1 # f1_3. An optional decimal point
    #f1 # f1_4. An arbitrary (but possibly zero) number of more digits
    #f1 # f1_5. A space (maybe more than one)
    #f1 # f1_6. The word 'million' or 'billion'"
    
    #form_one = r'\$\d+\.?\d*\s*[mb]ill?i?on'
    form_one = r'\$\d+\.?\d*\s*[mb]illion'

    # D2-11. Write a regular expression to match the three elements of "form_two" of the box office data.
    #   # "Create the Second Form ['$123,456,789' -- Module 8.3.10]
    #   # Next ... the numbers of our second form (f2 comments below) ... pattern match string will
    #   # include the following (three) elements:
    #   #
    #f2 # f2_1. A dollar sign
    #f2 # f2_2. A group of one to three digits
    #f2 # f2_3. At least one group starting with a comma and followed by exactly three digits    
    
    form_two = r'\$\d{1,3}(?:,\d{3})+'

    # D2-12. Add the parse_dollars function.
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

        # if input is of the form $###.# million
        #if re.match(r'\$\s*\d+\.?\d*\s*mill?i?on', s, flags=re.I):
        if re.match(r'\$\s*\d+\.?\d*\s*million', s, flags=re.I):

            # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]', '', s)

            # convert to float and multiply by a million
            value = float(s) * 10**6

            # return value
            return value

        # if input is of the form $###.# billion
        #elif re.match(r'\$\s*\d+\.?\d*\s*bill?i?on', s, flags=re.I):
        elif re.match(r'\$\s*\d+\.?\d*\s*billion', s, flags=re.I):

            # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]', '', s)

            # convert to float and multiply by a billion
            value = float(s) * 10**9

            # return value
            return value

        # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.I):
    
            # remove dollar sign and commas
            s = re.sub('\$|,', '', s)

            # convert to float
            value = float(s)

            # return value
            return value

        # otherwise, return NaN
        else:
            return np.nan
    
    # D2-13. Clean the box office column in the wiki_movies_df DataFrame.
    #wiki_movies_df['Box office'] = \
    wiki_movies_df['box_office'] = \
              box_office.str.extract(f'({form_one}|{form_two})',
                         flags=re.I)[0].apply(parse_dollars)

    # Drop the newly converted original (now no longer needed) Box Office column
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
    
    # D2-14. Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_movies_df['Budget'].dropna()
    
    #wiki_movies_df['Budget'] = \
    wiki_movies_df['budget'] = \
              budget.str.extract(f'({form_one}|{form_two})',
                           flags=re.I)[0].apply(parse_dollars)

#     # Drop the newly converted original (now no longer needed) Budget column
#     wiki_movies_df.drop('Budget', axis=1, inplace=True)

    # D2-15. Clean the release date column in the wiki_movies_df DataFrame.
    # "The (four) forms we'll be parsing are:"
    #  -p1} Full month name, one- to two-digit day, four-digit year
    #      (i.e., January 1, 2000)
    #  -p2} Four-digit year, two-digit month, two-digit day, with
    #      any separator (i.e., 2000-01-01)
    #  -p3} Full month name, four-digit year (i.e., January 2000)
    #  -p4} Four-digit year
    #
    date_form_one = \
        r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[0123]\d'
    date_form_three = \
        r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'    

    # parse the dates with built-in Pandas method
    # ..First, make a variable that holds the non-null values of
    # Release date in the DataFrame, converting lists to strings:"
    release_date = \
        wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

# commenting-out code line below and changing to code line below it so that KeyError
# from running lines 336-337 of this cell stops giving KeyError
#     wiki_movies_df['Release date'] = pd.to_datetime(
    wiki_movies_df['release_date'] = pd.to_datetime(
        release_date.str.extract(
            f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0],
            infer_datetime_format=True)
    
    # D2-16. Clean the running time column in the wiki_movies_df DataFrame.
    # Check for data and parse
    running_time = wiki_movies_df['Running time'].dropna().apply(
            lambda x: ' '.join(x) if type(x) == list else x)
    
    # Extract the digits, allowing for alternate patterns
    running_time_extract = running_time.str.extract(
            r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    
    # Convert strings to numeric; coercing the errors to turn any empty
    # strings into Not a Number (NaN), then using `fillna()` to change
    # all the NaNs to zeros."
    running_time_extract = running_time_extract.apply(
            lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

    # Convert the capture groups to minutes ([0][1] are hours and minutes,
    # respectively; capture group [2] is strictly minutes)
    wiki_movies_df['running_time'] = running_time_extract.\
        apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    
    # Drop the now converted original `Running time` column from the dataset:
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
    
    # ---Code in the cell below here up to return statement is for cleaning Kaggle metadata---
    # change comment immediately above to one immediately below...
    # ---Code in the cell below here up to '===' sign lines is for cleaning Kaggle metadata---
    #
    # As specified in Deliverable 3 for the Module 8 Challenge text,
    # in this section of the notebook, will be using "knowledge of Python, Pandas, the ETL
    # process, and code refactoring (to) extract and transform the Kaggle metadata and
    # MovieLens rating data, then convert the transformed data into separate DataFrames.
    # (will then) merge the Kaggle metadata DataFrame with the Wikipedia movies DataFrame to
    # create the `movies_df` DataFrame. Finally, (will) merge the MovieLens rating data
    # DataFrame with the `movies_df` DataFrame to create the `movies_with_ratings_df`."
        
    # As inspected, six columns need to converted from 'object' data type as follows
    # (in the kaggle_metadata dataframe): 
    # .  id           -->  'numeric'
    # .  popularity   -->  'numeric'
    # .  release_date -->  'datetime'
    # .  adult        -->  'Boolean'
    # .  video        -->  'Boolean'
    # .  budget       -->  'numeric'
    
#     # Were to use following code line to remove bad data
#     # (e.g., text data in Boolean column), but will use the uncommented
#     # code line below this one instead (as will be removing--dropping--
#     # the 'adult' column).
#     kaggle_metadata[~kaggle_metadata['adult'].isin(['True','False'])]
    #
    # "The following code will keep rows where the adult column is
    # `False`, and then drop the adult column." ...
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop(
                                  'adult',axis='columns')
    
    # convert 'video' values to Boolean, assign back to column of DataFrame:
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
    
    # "For the numeric columns, we can just use the `to_numeric()`
    # method from Pandas. We'll make sure the `errors=` argument
    # is set to `raise`, so we'll know if there's any data that
    # can't be converted to numbers." --commenting from Mod. 8.3.12
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)

    kaggle_metadata['id'] = pd.to_numeric(
                            kaggle_metadata['id'], errors='raise')

    kaggle_metadata['popularity'] = pd.to_numeric(
                                    kaggle_metadata['popularity'], errors='raise')
    
    # (The above three lines of) "code above runs without errors,
    # so everything converted fine." --commenting from Mod. 8.3.12
    #
    # "Since `release_date` is in a standard format, `to_datetime()`
    # will convert it without any fuss." --commenting from Mod. 8.3.12
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

    # === Code in the cell below here, continuing down to "+++" sign lines, is to merge ===
    # === as specified in comments below, the `wiki_movies_df` and `kaggle_metadata` ===
    # === === === === === === === === === DataFrames === === === === === === === === ===

    # merge the `wiki_movies_df` and `kaggle_metadata` DataFrames, then name the new
    # DataFrame, `movies_df` ...
    # (note: using Pandas.merge --will assign to a new DataFrame, specifying left and
    # right objects to be joined database-style; as opposed to Pandas.DataFrame.merge
    # --where is DataFrame is left object and right object is passed as function argu-
    # ment to be database-style joined to DataFrame) --see details in 'API reference'
    # at pandas.pydata.org website documentation
    movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id',\
                         suffixes=['_wiki','_kaggle'])

    # input(movies_df[['title_wiki','title_kaggle']])

    # input(f"rows in ...movies_df: {len(movies_df)},\
    #       \n...wiki_movies_df['imdb_id']: {len(wiki_movies_df['imdb_id'].value_counts())},\
    #       \n...kaggle_metadata['imdb_id']: {len(kaggle_metadata['imdb_id'].value_counts())}")
    
    # note: after the join, the data still needs to be "cleaned up a bit, especially where
    # Kaggle and Wikipedia data overlap -- redundant columns" -- Module 8.4.1

    # ***Below*** this next code line, will be checking seven cases of modifications to be
    # made to `movies_df` DataFrame
    # but for this next code line, will be dropping a row with garbled data from Module 8.4.1
    # (This title was row index 3607; in the module found with following code--commented-out)...
    # (where *"The Holiday"* got somehow merged with *"From Here to Eternity"*)
    # movies_df[(movies_df['release_date_wiki'] > '1996-01-01') &\
    #          (movies_df['release_date_kaggle'] < '1965-01-01')].index
    #
#     try:
#         input(f'{movies_df.columns.to_list()}')
#         movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & \
#                                         (movies_df['release_date_kaggle'] < '1965-01-01')].index
#     except KeyError as err:
#         print(err)

    # input(movies_df[(movies_df['release_date_wiki'] > '1996-01-01') &\
    #               (movies_df['release_date_kaggle'] < '1965-01-01')].index)
    # code to drop that row is...
    # movies_df = movies_df.drop(movies_df[(movies_df['release_date_wiki'] > '1996-01-01') & \
    #                                     (movies_df['release_date_kaggle'] < '1965-01-01')].index)    
    
    # summary table is below of resolution to overlapping/redundant columns...
    # (determined by review of scatter plots -- except for datetime which used
    #  '.'-style line chart plot)
# Mod  Wikipedia column name  Kaggle column name     Resolution description
# ---  ---------------------  ---------------------  --------------------------------
#  A   title_wiki             title_kaggle           Drop Wikipedia.
#  B   running_time           runtime                Keep Kaggle; fill-in zeros with Wikipedia data.
#  C   budget_wiki            budget_kaggle           "     "        "      "    "      "       "
#  D   box_office             revenue                 "     "        "      "    "      "       "
#  E   release_date_wiki      release_date_kaggle    Drop Wikipedia.
#  F   Language               original_language       "       "
#  G   Production company(s)  production_companies    "       "

    # Modifications 'A', 'E', 'F', and 'G'.
    movies_df.drop(columns=['title_wiki', 'release_date_wiki', 'Language',\
                            'Production company(s)'], inplace=True)
    
    # for expediency (as instructed), define function below to perform remaining modifications
    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(
            lambda row: row[wiki_column] if row[kaggle_column] == 0
                        else row[kaggle_column], axis=1)
        df.drop(columns=wiki_column, inplace=True)
    
    # Now, can run the function for the three remaining column modifications ('B', 'C', and 'D')
    fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
    fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
    fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')

    # input(movies_df)
    
    # for loop for checking for and removing if any columns with only one value,
    # remembering that for `value_counts()` to work, must convert lists to tuples...
    for col in movies_df.columns:
        lists_to_tuples = lambda x: tuple(x) if type(x) == list else x
        value_counts = movies_df[col].apply(lists_to_tuples).value_counts(dropna=False)
        num_values = len(value_counts)
        if num_values == 1:
            movies_df.drop(col, axis=1, inplace=True)
            # print(f"movies_df['{col}'] uninformative (all same value of '{movies_df[col][0]}');...dropping.")
            # response = input("Confirm drop column? '[y]/n'")
            # # print(type(response))            
            # if (len(response) == 0) | (response != 'n'):
            #     try:
            #         movies_df.drop(col, axis=1, inplace=True)
            #     # # response = input("Confirm drop column? '[y]/n'")
            #     # # if (str(response) == "") or (str(response) != 'n'):
            #     # if str(input("Confirm drop column? '[y]/n'")) != 'n':
            #     #     movies_df.drop(col, axis=1, inplace=True)
            #     # else:
            #     #     print(f"movies_df['{col}'] column NOT dropped.")
            #     except KeyError:
            #         print("KeyError occurred.")

    # "reorder ... columns roughly in groups (easier to read)" -- e.g., like this:
    # r1. Identifying information (IDs, titles, URLs, etc.)                         
    # r2. Quantitative facts (runtime, budget, revenue, etc.)
    # r3. Qualitative facts (genres, languages, country, etc.)
    # r4. Business data (production conmpanies, distributors, etc.)
    # r5. People (producers, director, cast, writers, etc.)
                         
    # prior to rename of columns, use `.loc` to *reorder* the columns (i.e., ***instead of***
    # passing list of column names like `movies_df = movies_df[['imdb_id', 'title_kaggle', ...]]`
    # to the indexing operator) so as to avoid receiving a SettingWithCopyWarning.
    # Module 8.4.1 note: "Don't panic! (if receive the message) This isn't an error, so your code
    # will continue to work, but it is a warning that (the) code may not behave as expect(ed). In
    # this case, your code will work fine, but for best practices, use `.loc` instead to avoid
    # this warning."
    movies_df = movies_df.loc[:, ['imdb_id', 'id', 'title_kaggle', 'original_title', 'tagline',
                                  'belongs_to_collection', 'url', 'imdb_link', 'runtime',
                                  'budget_kaggle', 'revenue', 'release_date_kaggle',
                                  'popularity', 'vote_average', 'vote_count', 'genres',
                                  'original_language', 'overview', 'spoken_languages',
                                  'Country', 'production_companies', 'production_countries',
                                  'Distributor', 'Producer(s)', 'Director', 'Starring',
                                  'Cinematography', 'Editor(s)', 'Writer(s)', 'Composer(s)',
                                  'Based on'
                                 ]]
    # after reordering with `.loc` as noted in comment (to avoid receiving a
    # SettingWithCopyWarning) and coded above, "rename the columns to be consistent"
    #                                                                  --Module 8.4.1
    movies_df.rename({'id':'kaggle_id',
                      'title_kaggle':'title',
                      'url':'wikipedia_url',
                      'budget_kaggle':'budget',
                      'release_date_kaggle':'release_date',
                      'Country':'country',
                      'Distributor':'distributor',
                      'Producer(s)':'producers',
                      'Director':'director',
                      'Starring':'starring',
                      'Cinematography':'cinematography',
                      'Editor(s)':'editors',
                      'Writer(s)':'writers',
                      'Composer(s)':'composers',
                      'Based on':'based_on'
                     }, axis='columns', inplace=True)
    

    # +++ Code in the cell below here, down to return statement, is to merge as specified +++    
    # +++ in comments below, the metadata from the `movies_df` and `ratings` DataFrames into +++
    # +++ +++ a new DataFrame `movies_with_ratings_df`, then clean the new DataFrame +++ +++
   
    # in the Module, specifically in 8.3.12, we reviewed the 'timestamp' column from the ratings
    # csv data file. we will see later on in this cell, that do not use this 'timestamp' column
    # for our own analysis purposes; however, as will be storing rating data as own table in SQL,
    # will (need to) convert that timestamp data to a datetime data type. "From the MovieLens
    # documentation, the timestamp is the number of seconds since midnight of January 1, 1970
    # (known as the Unix **epoch**.)"
    ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
    
    # "a ... useful summary is (for ratings data file).. to count how many times a movie
    # received a given rating. This way, someone who wants to calculate statistics for the
    # dataset (has) all the information they need ... easy enough to do. Plus, (can)
    # calculate statistics ... without having to work with a dataset containing 26-
    # million rows. ..First, ... use a `groupby` on the 'movieID' and 'rating'
    # columns and take the count for each group." --comments quoted from Mod. 8.4.2
    # code looks like following line...(commented out as will be completed lower below)
#     rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count()
    #
    # (then next, and as it turns out arbitrarily -- as "timestamp" column has the
    # same information; "so ...could use either one.")
    # "rename the 'userID' column to 'count.'" --from Module 8.4.2
    # (also commented out below as will be completed below yet again another time)
#     rating_counts = ratings.groupby(['movieId','rating'], as_index=False).\
#                         count.rename({'userID':'count'}, axis=1)
    #
    # now, for a "magical part" whereby "pivot this data so that `movieId` is the
    # index, the columns will be all the rating values, and the rows will be the
    # counts for each rating value." --Module 8.4.2
    rating_counts = ratings.groupby(['movieId','rating'], as_index=False).\
                    count().rename({'userId':'count'}, axis=1).\
                    pivot(index='movieId', columns='rating', values='count')
    #
    # and then so that the "columns (are) easier to understand ... (rename with a)
    # prepend (of) `rating_` to each column with a list comprehension:" --Module 8.4.2
    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]
    
    # Now we can merge the rating counts into `movies_df` (note use a left merge, since
    # we want to keep everything in `movies_df`:) --Module 8.4.2
    # (note: using Pandas.merge --will assign to a new DataFrame, specifying left and
    # right objects to be joined database-style; as opposed to Pandas.DataFrame.merge
    # --where is DataFrame is left object and right object is passed as function argu-
    # ment to be database-style joined to DataFrame) --see details in 'API reference'
    # at pandas.pydata.org website documentation
    # perform the merge of the DataFrames
    movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id',
                                     right_index=True, how='left')
    #
    # Clean the `movies_with_ratings_df` DataFrame by filling in missing values with zeros
    movies_with_ratings_df[rating_counts.columns] =\
                         movies_with_ratings_df[rating_counts.columns].fillna(0)
    
#     return wiki_movies_df, movies_with_ratings_df, movies_df
    return wiki_movies_df, movies_with_ratings_df, movies_df

In [4]:
# 10. Create the path to your file directory and variables for the three files.
file_dir = "../../../DataBootcamp/Mod_8/"
# The Wikipedia data
#wiki_file = f'{file_dir}/wikipedia_movies.json'
wiki_file = f'{file_dir}/wikipedia-movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [5]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

In [6]:
ratings_file.columns.to_list()

['imdb_id',
 'kaggle_id',
 'title',
 'original_title',
 'tagline',
 'belongs_to_collection',
 'wikipedia_url',
 'imdb_link',
 'runtime',
 'budget',
 'revenue',
 'release_date',
 'popularity',
 'vote_average',
 'vote_count',
 'genres',
 'original_language',
 'overview',
 'spoken_languages',
 'country',
 'production_companies',
 'production_countries',
 'distributor',
 'producers',
 'director',
 'starring',
 'cinematography',
 'editors',
 'writers',
 'composers',
 'based_on']

In [7]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [8]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Country,Language,...,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",United States,English,...,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",United States,English,...,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",United States,"[English, Lao]",...,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005,57718089.0,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",United States,English,...,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012,7331647.0,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,"December 19, 1990",US,English,...,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018,6939946.0,25000000.0,1990-12-19,95.0


In [9]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df.head()

,imdb_id,kaggle_id,title,original_title,tagline,belongs_to_collection,wikipedia_url,imdb_link,runtime,budget,...,rating_0.5,rating_1.0,rating_1.5,rating_2.0,rating_2.5,rating_3.0,rating_3.5,rating_4.0,rating_4.5,rating_5.0
0,tt0098987,9548,The Adventures of Ford Fairlane,The Adventures of Ford Fairlane,Kojak. Columbo. Dirty Harry. Wimps.,NaN,https://en.wikipedia.org/wiki/The_Adventures_o...,https://www.imdb.com/title/tt0098987/,104.0,49000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,tt0098994,25501,"After Dark, My Sweet","After Dark, My Sweet",All they risked was everything.,NaN,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",https://www.imdb.com/title/tt0098994/,114.0,6000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,tt0099005,11856,Air America,Air America,The few. The proud. The totally insane.,NaN,https://en.wikipedia.org/wiki/Air_America_(film),https://www.imdb.com/title/tt0099005/,112.0,35000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,tt0099012,8217,Alice,Alice,NaN,NaN,https://en.wikipedia.org/wiki/Alice_(1990_film),https://www.imdb.com/title/tt0099012/,102.0,12000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,tt0099018,25943,Almost an Angel,Almost an Angel,Who does he think he is?,NaN,https://en.wikipedia.org/wiki/Almost_an_Angel,https://www.imdb.com/title/tt0099018/,95.0,25000000.0,...,3.0,0.0,3.0,2.0,5.0,26.0,37.0,46.0,16.0,11.0


In [10]:
# 15. Check the movies_df DataFrame. 
movies_df.head()

,imdb_id,kaggle_id,title,original_title,tagline,belongs_to_collection,wikipedia_url,imdb_link,runtime,budget,...,production_countries,distributor,producers,director,starring,cinematography,editors,writers,composers,based_on
0,tt0098987,9548,The Adventures of Ford Fairlane,The Adventures of Ford Fairlane,Kojak. Columbo. Dirty Harry. Wimps.,NaN,https://en.wikipedia.org/wiki/The_Adventures_o...,https://www.imdb.com/title/tt0098987/,104.0,49000000.0,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",20th Century Fox,"[Steve Perry, Joel Silver]",Renny Harlin,"[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,Michael Tronick,"[David Arnott, James Cappe]","[Cliff Eidelman, Yello]","[Characters, by Rex Weiner]"
1,tt0098994,25501,"After Dark, My Sweet","After Dark, My Sweet",All they risked was everything.,NaN,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",https://www.imdb.com/title/tt0098994/,114.0,6000000.0,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",Avenue Pictures,"[Ric Kidney, Robert Redlin]",James Foley,"[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,Howard E. Smith,"[James Foley, Robert Redlin]",Maurice Jarre,"[the novel, After Dark, My Sweet, by, Jim Thom..."
2,tt0099005,11856,Air America,Air America,The few. The proud. The totally insane.,NaN,https://en.wikipedia.org/wiki/Air_America_(film),https://www.imdb.com/title/tt0099005/,112.0,35000000.0,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",TriStar Pictures,Daniel Melnick,Roger Spottiswoode,"[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[John Bloom, Lois Freeman-Fox]","[John Eskow, Richard Rush]",Charles Gross,"[Air America, by, Christopher Robbins]"
3,tt0099012,8217,Alice,Alice,NaN,NaN,https://en.wikipedia.org/wiki/Alice_(1990_film),https://www.imdb.com/title/tt0099012/,102.0,12000000.0,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",Orion Pictures,Robert Greenhut,Woody Allen,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,Susan E. Morse,Woody Allen,NaN,NaN
4,tt0099018,25943,Almost an Angel,Almost an Angel,Who does he think he is?,NaN,https://en.wikipedia.org/wiki/Almost_an_Angel,https://www.imdb.com/title/tt0099018/,95.0,25000000.0,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",Paramount Pictures,John Cornell,John Cornell,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,David Stiven,Paul Hogan,Maurice Jarre,NaN
